# Importing Data from MySQL Database

### Import Necessary module and Library

In [1]:
import pymysql
import pandas as pd
import os

### Set up database connection

In [2]:
conn = pymysql.connect(host= os.environ.get('SQLHOSTID'),
                      port = 3306,
                      user = os.environ.get('SQLUSERID'),
                      passwd = os.environ.get('SQLPASSWORD'),
                      db = 'financial')

### Import data

In [3]:
df = pd.read_sql('''select account.account_id, disp_O.client_id as owner_client_id, disp_O.disp_id 
as owner_disp_id, client.gender, disp_num.num_acct_users, client.birth_date as owner_birth_date, account.frequency as statement_freq, card.type as card_type, card.issued as card_issue_date, account.date as acct_start_date, loan.loan_id, loan.date as loan_date, loan.amount as loan_amt, payments.total_paid, loan.duration as loan_duration_mos, payments.total_payments, loan.payments as monthly_payments, loan.status as loan_status, account.district_id, district.A2, district.A3, district.A4, district.A5, district.A6, district.A7, district.A8, district.A9, district.A10, district.A11, district.A12, district.A13, district.A14, district.A15, district.A16
from loan
left join account on loan.account_id = account.account_id
left join district on account.district_id = district.district_id
left join (select account_id, sum(amount) as total_paid, count(date) as total_payments
from financial.trans
            where k_symbol like 'UVER'
            group by account_id) as payments on account.account_id = payments.account_id
left join (select * from disp
            where disp.type like 'O%') as disp_O on loan.account_id = disp_O.account_id
left join client on disp_O.client_id = client.client_id
left join card on disp_O.disp_id = card.disp_id
left join (select account_id, count(client_id) as num_acct_users
from disp group by account_id) as disp_num on loan.account_id = disp_num.account_id''', conn)

In [4]:
df.head()

,account_id,owner_client_id,owner_disp_id,gender,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,...,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,2,2,M,2,1945-02-04,POPLATEK MESICNE,None,None,1993-02-26,...,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,19,25,25,F,1,1939-04-23,POPLATEK MESICNE,None,None,1995-04-07,...,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,25,31,31,M,1,1962-02-09,POPLATEK MESICNE,None,None,1996-07-28,...,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,37,45,45,M,1,1952-08-26,POPLATEK MESICNE,None,None,1997-08-18,...,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,38,46,46,F,1,1940-01-30,POPLATEK TYDNE,None,None,1997-08-08,...,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099


In [5]:
#Save data

df.to_csv('raw_dataset.csv', index = False)

### Close Connection

In [6]:
conn.close()